In [1]:
import serial
import time

# Specify the USB port (e.g., '/dev/ttyUSB0', '/dev/ttyACM0')
port = "/dev/ttyACM0"
baudrate = 1000000  # Common baudrate for Feetech servos

# Open serial connection
ser = serial.Serial(port, baudrate, timeout=1)
time.sleep(0.1)

In [2]:
def ping_servo(ser, servo_id):
    """Ping a specific servo ID to check if it responds"""
    # Ping command (0x01 = Ping)
    packet = [0xFF, 0xFF, servo_id, 0x02, 0x01]
    checksum = ~sum(packet[2:]) & 0xFF
    packet.append(checksum)

    # Clear buffer
    ser.reset_input_buffer()

    # Send ping command
    ser.write(bytes(packet))
    time.sleep(0.01)

    # Read response
    response = ser.read(6)  # Expected response length for ping
    if len(response) >= 6:
        # Response format: [0xFF, 0xFF, ID, Length, Error, Checksum]
        if response[0] == 0xFF and response[1] == 0xFF:
            return True
    return False


def scan_servos(ser, id_range=range(0, 254)):
    """Scan for all connected servos and return their IDs"""
    print("Scanning for servos...")
    found_servos = []

    for servo_id in id_range:
        if ping_servo(ser, servo_id):
            print(f"  Found servo with ID: {servo_id}")
            found_servos.append(servo_id)

    if not found_servos:
        print("  No servos found!")
    else:
        print(f"\nTotal servos found: {len(found_servos)}")

    return found_servos


# Feetech SCS servo protocol: Read ID command
# Packet format: [0xFF, 0xFF, ID, Length, Instruction, Address, Length_to_read, Checksum]
def read_servo_id(ser, servo_id):
    """Read and print the current servo ID"""
    # Read ID command (0x02 = Read, 0x05 = ID address, 0x01 = read 1 byte)
    packet = [0xFF, 0xFF, servo_id, 0x04, 0x02, 0x05, 0x01]
    checksum = ~sum(packet[2:]) & 0xFF
    packet.append(checksum)

    # Clear buffer
    ser.reset_input_buffer()

    # Send read command
    ser.write(bytes(packet))
    time.sleep(0.05)

    # Read response
    response = ser.read(8)  # Expected response length
    if len(response) >= 6:
        # Response format: [0xFF, 0xFF, ID, Length, Error, Data, Checksum]
        if response[0] == 0xFF and response[1] == 0xFF:
            read_id = response[5]  # The data byte contains the ID
            print(f"Current servo ID: {read_id}")
            return read_id
        else:
            print("Invalid response header")
            return None
    else:
        print(f"No response or incomplete response (got {len(response)} bytes)")
        return None


# Feetech SCS servo protocol: Set ID command
# Packet format: [0xFF, 0xFF, ID, Length, Instruction, Address, Data, Checksum]
def set_servo_id(ser, old_id, new_id):
    """Set a new servo ID"""
    # Write ID command (0x03 = Write, 0x05 = ID address)
    packet = [0xFF, 0xFF, old_id, 0x04, 0x03, 0x05, new_id]
    checksum = ~sum(packet[2:]) & 0xFF
    packet.append(checksum)
    ser.write(bytes(packet))
    time.sleep(0.1)
    print(f"Set servo ID from {old_id} to {new_id}")

In [3]:
# Scan for all connected servos
found_ids = scan_servos(ser)

if found_ids:
    print(f"\nConnected servo IDs: {found_ids}")

    # If you want to change an ID, uncomment and modify these lines:
    # old_id = found_ids[0]  # The current ID
    # new_id = 2  # The new ID you want to set
    # set_servo_id(ser, old_id, new_id)
    # print("Verifying new ID...")
    # scan_servos(ser)  # Scan again to verify

ser.close()

Scanning for servos...
  Found servo with ID: 6


KeyboardInterrupt: 

In [7]:
set_servo_id(ser, 2, 6)

Set servo ID from 2 to 6


In [8]:
scan_servos(ser)

Scanning for servos...
  Found servo with ID: 6


KeyboardInterrupt: 

In [9]:
ser.close()